In [2]:
#Operation Class
## Input Nodes
## Output Nodes
## Global Default Graph Variable
## Comput
###Overwritten by extended class

In [3]:
# Graph - A Global Variable
#### Constant (N1) weight 1
#### Constant (N2) weight 2
######## Pointing to Operation (N3)   (Adding?)

## N1 = 1
## N2 = 2
## Operation (N3)
## Output = 3

## Operation

In [26]:
class Operation():
    
    def __init__(self, input_nodes=[]):
        
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        # Grab node, grab output for particular node, and append self
        # Reference to current operation
        for nodes in input_nodes:
            nodes.output_nodes.append(self)
        
        _default_graph.operations.append(self)
        
    def comput(self):
        pass

In [27]:
class add(Operation):
    
    def __init__(self, x, y):
        #X & Y are inputs
        super().__init__([x,y])
        
    #Will overwrite compute function in Operation    
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var

In [28]:
class multiply(Operation):
    
    def __init__(self, x, y):
        #X & Y are inputs
        super().__init__([x,y])
        
    #Will overwrite compute function in Operation    
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var * y_var

In [29]:
class matmul(Operation):
    
    def __init__(self, x, y):
        #X & Y are inputs
        super().__init__([x,y])
        
    #Will overwrite compute function in Operation    
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var.dot(y_var)

In [30]:
class Placeholder():
    
    def __init__(self):
        
        self.output_nodes = []
        _default_graph.placeholders.append(self)

In [31]:
class Variable():
    
    def __init__(self, initial_value=None):
        
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

In [32]:
#Graph object contains place holder and variables
#Variable are changeable parameters within the graph
#Placeholders are empty nodes, waiting for data information
#Operations are .... operations....

class Graph():
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        
        global _default_graph
        
        _default_graph = self
        

z = Ax + b

A = 10

b = 1

z = 10x + 1

In [33]:
g = Graph()

In [34]:
g.set_as_default()

In [35]:
A = Variable(10)

In [36]:
b = Variable(1)

In [37]:
x = Placeholder()

In [38]:
y = multiply(A, x)

In [39]:
z = add(y,b)

In [40]:
def traverse_postorder(operation):
    """
    PostOrder Traversal of Nodes.
    
    Makes sure computations are done in the correct order (Ax first, Ax+b second).
    """
    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
        
    recurse(operation)
    
    return nodes_postorder

In [41]:
class Session():
    
    def run(self, operation, feed_dict={}):
        """
        feed dictionary is dictionary mapping place holders to input values
        """
        
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            
            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else:
                # OPERATION
                node.inputs = [input_node.output for input_node in node.input_node]
                
                node.output = node.compute(*node.inputs) #args
                
            if type(node.output) == list:
                
                node.output = np.array(node.output)
                
        return operation.output